In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt



In [0]:
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
class_names_label = {
    'buildings': 0,
    'forest': 1,
    'glacier': 2,
    'mountain': 3,
    'sea': 4,
    'street': 5
    }
n_classes = 6


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def load_data():
    datasets = ['seg_train/seg_train', 'seg_test/seg_test']
    size = (150, 150)
    output = []
    for dataset in datasets:
        directory = "gdrive/My Drive/team Errorist/input/intel-image-classification/" + dataset
        images = []
        labels = []
        for folder in os.listdir(directory):
            curr_label = class_names_label[folder]
            for file in os.listdir(directory + "/" + folder):
                img_path = directory + "/" + folder + "/" + file
                curr_img = cv2.imread(img_path)
                curr_img = cv2.resize(curr_img, size)
                images.append(curr_img)
                labels.append(curr_label)
        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')

        images = images / 255

        output.append((images, labels))

    return output

In [0]:
(train_images, train_labels), (test_images, test_labels) = load_data()


In [0]:
trainimages = np.save('gdrive/My Drive/team Errorist/input/trainimages', train_images)
trainlabels = np.save('gdrive/My Drive/team Errorist/input/trainlabels', train_labels)
testimages = np.save('gdrive/My Drive/team Errorist/input/testimages', test_images)
testlabels = np.save('gdrive/My Drive/team Errorist/input/testlabels', test_labels)

def train_val_split(X, y, val_ratio, random_seed):
    np.random.seed(random_seed)
    shuffled = np.random.permutation(len(X))
    val_size = int(len(X) * val_ratio)
    X_train = X[shuffled[val_size:]]
    X_val = X[shuffled[:val_size]]
    y_train = y[shuffled[val_size:]]
    y_val = y[shuffled[:val_size]]

    return X_train, X_val, y_train, y_val

In [0]:
X_train, X_val, y_train, y_val = train_val_split(train_images,
                                                 train_labels,
                                                 val_ratio=0.2,
                                                 random_seed=42)

In [0]:
print(X_train.shape)

(11241, 150, 150, 3)


In [0]:
def vgg_16(input_shape, label_num):
    X_input = tf.keras.layers.Input(shape=input_shape)

    X = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3),
                               padding='same')(X_input)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(X)

    X = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(X)

    X = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(X)

    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(X)

    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3),
                               padding='same')(X)
    X = tf.keras.layers.ELU()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(X)

    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(units=4096, activation=tf.nn.elu)(X)
    X = tf.keras.layers.Dropout(rate=0.5)(X)
    X = tf.keras.layers.Dense(units=4096, activation=tf.nn.elu)(X)
    X = tf.keras.layers.Dropout(rate=0.5)(X)
    X = tf.keras.layers.Dense(units=label_num, activation=tf.nn.softmax)(X)

    model = tf.keras.models.Model(inputs=X_input, outputs=X)
    return model

In [0]:
input_shape = (150, 150, 3)
label_num = n_classes
model = vgg_16(input_shape, label_num)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                        loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      mode='min',
                                      verbose=1,
                                      patience=10)
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5',
                                        monitor='val_accuracy',
                                        mode='max',
                                        verbose=1,
                                        save_best_only=True)

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(X_train)

In [0]:
batch_size = 128
hist = model.fit_generator(datagen.flow(X_train,
                                        y_train,
                                        batch_size=batch_size),
                           epochs=80,
                           validation_data=(X_val, y_val),
                           steps_per_epoch=X_train.shape[0] // batch_size,
                           callbacks=[es, mc])


Epoch 1/80
87/87 [==============================] - 61s 702ms/step - loss: 0.9753 - acc: 0.6297 - val_loss: 0.8588 - val_acc: 0.6826
Epoch 2/80
87/87 [==============================] - 59s 674ms/step - loss: 0.8477 - acc: 0.6851 - val_loss: 0.7489 - val_acc: 0.7395
Epoch 3/80
87/87 [==============================] - 59s 679ms/step - loss: 0.8006 - acc: 0.7025 - val_loss: 0.8980 - val_acc: 0.7004
Epoch 4/80
87/87 [==============================] - 58s 667ms/step - loss: 0.7260 - acc: 0.7375 - val_loss: 0.7235 - val_acc: 0.7406
Epoch 5/80
87/87 [==============================] - 58s 663ms/step - loss: 0.6578 - acc: 0.7597 - val_loss: 0.5785 - val_acc: 0.8064
Epoch 6/80
87/87 [==============================] - 57s 654ms/step - loss: 0.6278 - acc: 0.7772 - val_loss: 0.5694 - val_acc: 0.8135
Epoch 7/80
87/87 [==============================] - 57s 659ms/step - loss: 0.5854 - acc: 0.7910 - val_loss: 0.6485 - val_acc: 0.7833
Epoch 8/80
87/87 [==============================] - 57s 651ms/step - 

In [0]:
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("gdrive/My Drive/team Errorist/input/model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
score = model.evaluate(test_images, test_labels)


3000/3000 [==============================] - 5s 2ms/sample - loss: 0.5279 - acc: 0.8413


In [0]:
model.save('gdrive/My Drive/team Errorist/input/Final.h5')